### 대리점의 매출 이력과 고객 정보 데이터를 이용한 데이터 가공

- 대리점 데이터는 <u>시스템에 의해 관리되는 쇼핑몰 사이트</u>와는 다르게 사람의 손을 탑니다.               
그래서 날짜 입력 실수나 데이터 누락 등'오류'가 많이 포함되어 있습니다.             
이런 **'지저분한' 데이터를 다루는 연습으로 대리점 데이터가 적당**하다고 할 수 있습니다.               
              
              
- 대리점 이외의 비즈니스 현장에서도 엑셀(Excel)처럼 수작업으로 입력하는 경우가 많습니다.                 
손으로 입력한 데이터는 기계적인 입력 확인이 이루어지지 않기 때문에           
데이터가 점점 '지저분'해지고 그대로는 데이터 분석에 활용할 수 없습니다.                   
                    
                   
- 그 외에도, 각각 다른 부서에서 수집된 데이터를 다루는 경우에는                
각기 독자적인 시스템으로 데이터가 관리되므로 그것들을 통일하는 것 또한 쉽지 않습니다.            

고객의 소리)
우리 회사에서는 고객 정보를 엑셀로 관리합니다. 장사는 잘되고 대리점의 매출도 안정적입니다. 데이터가 풍부할 때 데이터 분석을 해두면 여러 가지 발견을 할 수 있지 않을까 합니다. 시험 삼아 데이터 분석을 부탁드립니다.

전제 조건)

상품 : A ~ Z 까지 26개의 상품                         
데이터 : 매출 이력 & 고객 정보 데이터를 담당 사원이 시스템에 '직접 입력'                         

 * 집계 기간 중 상품 단가 변동 X                
 * 매출 이력은 시스템에서 csv파일로 출력                          
 * 고객 정보는 대리점 '관리자가 주별로 집계'해서 엑셀로 관리
<n>

</n>

 uriage.csv : 매출 이력(기간은 2019.01 ~ 2019.07)                   
 kokyaku_daicho.xlsx : 대리점에서 관리하는 고객 정보

## 데이터 정제

In [455]:
import pandas as pd

sale_df = pd.read_csv('C:/Users/LGCARE/Desktop/개인/Study_DataTechniques/01. DataProcess/data/2장/uriage.csv')
customer_df = pd.read_excel('C:/Users/LGCARE/Desktop/개인/Study_DataTechniques/01. DataProcess/data/2장/kokyaku_daicho.xlsx',
                            )

# sale_df.info()
print(sale_df.head())
print()

# customer_df.info()
print(customer_df.head())

# item_name, item_price에 결측치, 오류 존재 > '데이터의 정합성에 문제가 있다'


      purchase_date item_name  item_price customer_name
0  2019-06-13 18:02       상품A       100.0           김가온
1  2019-07-13 13:05     상 품 S         NaN           김우찬
2  2019-05-11 19:42     상 품 a         NaN           김유찬
3  2019-02-12 23:40       상품Z      2600.0           김재현
4   2019-04-22 3:09       상품a         NaN           김강현

   고객이름  지역                  등록일
0  김 현성  H시  2018-01-04 00:00:00
1  김 도윤  E시                42782
2  김 지한  A시  2018-01-07 00:00:00
3  김 하윤  F시                42872
4  김 시온  E시                43127


> 데이터 오류의 예

- 날짜 
1) 2019-10-10
2) 2019/10/10
3) 10/10/2019
4) 2019년 10월 10일                 
: 같은 날짜라도 포맷이 다르면 다른 문자열 데이터가 됩니다.                               
  이런 오류를 자동으로 수정해주는 언어도 있지만 여러 가지 포맷이 섞여 있는 경우에는 주의해야 합니다.

이름
1) 김현성
2) 김 현성
3) 김현 성
4) 김 현성
5) 김현  성                    
: 공백 유무로 시스템은 다른 데이터로 인식합니다.                   
  입력 시 실수로 인해 같은 이름이 다른 이름이 됩니다.

-> 데이터의 오류를 해소하고 정합성을 보장하는 것은 데이터 분석의 기초가 되는 중요한 부분              
-> 정합성을 갖추기 위해서는 먼저 '데이터의 속성'이나 '의미'를 이해해야 합니다.              
   매출 이력에 purchase_date, item_name, item_price, customer_name이 잇는 것을 확인할 수 있습니다.                
   정합성의 확보는 우선 데이터의 오류를 파악하는 것부터 시작합니다.                  

In [456]:
sale_df['item_name'].head() # 공백, 대소문자
sale_df['item_price'].head() # 결측치

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

>> 오류가 있는 상태에서 상품별 월 매출 집계

In [457]:
sale_df['purchase_date'] = pd.to_datetime(sale_df['purchase_date'])
sale_df.dtypes

sale_df['purchase_month'] = sale_df['purchase_date'].dt.strftime('%Y%m')
sale_df['purchase_month'].head()

# 1) groupby
print(sale_df.groupby(['purchase_month', 'item_name'])[['item_price']].sum())

# 2) pivot_table
# size : 상품의 판매 건수
print(pd.pivot_table(data= sale_df, index= 'purchase_month', columns= 'item_name', aggfunc= 'size',
                     fill_value= 0)) # 'size' > values (X)

                          item_price
purchase_month item_name            
201901          상 품 n         1400.0
               상 품H            800.0
               상 품O           1500.0
               상 품P           1600.0
               상 품T           2000.0
...                              ...
201907         상품X           38400.0
               상품Y           27500.0
               상품i             900.0
               상품o            1500.0
               상품r            1800.0

[265 rows x 1 columns]
item_name        상 품 n   상품 E   상품 M   상품 P   상품 S   상품 W   상품 X   상품W  상  품O  \
purchase_month                                                                  
201901               1      0      0      0      0      0      0     0      0   
201902               0      0      0      0      0      0      1     0      0   
201903               0      1      1      1      0      0      0     0      0   
201904               0      0      0      0      0      0      0     1      0   
201905   

In [458]:
# 상품 S와 상품 s가 별도의 상품으로 집계
# 원래 26개의 상품이 99개의 상품으로 늘어남

print(pd.pivot_table(data= sale_df, values= 'item_price', index= 'purchase_month', columns= 'item_name', aggfunc= 'sum',
                     fill_value= 0))

item_name        상 품 n   상품 E    상품 M    상품 P    상품 S    상품 W    상품 X     상품W  \
purchase_month                                                                  
201901          1400.0    0.0     0.0     0.0     0.0     0.0     0.0     0.0   
201902             0.0    0.0     0.0     0.0     0.0     0.0  2400.0     0.0   
201903             0.0  500.0  1300.0  1600.0     0.0     0.0     0.0     0.0   
201904             0.0    0.0     0.0     0.0     0.0     0.0     0.0  2300.0   
201905             0.0    0.0     0.0     0.0  1900.0     0.0     0.0     0.0   
201906             0.0    0.0     0.0     0.0     0.0  2300.0     0.0     0.0   
201907             0.0    0.0     0.0     0.0     0.0     0.0     0.0     0.0   

item_name       상  품O   상  품Q  ...     상품k     상품l     상품o     상품p     상품r  \
purchase_month                 ...                                           
201901            0.0     0.0  ...  1100.0  1200.0  1500.0     0.0     0.0   
201902            0.0     0.0  ...  

>> 상품명 오류 수정

In [459]:
# 명목형

# sale_df.head() # 공백, 대소문자

# 현황 파악

sale_df['item_name'].unique()

# sale_df['item_name'] = sale_df['item_name'].apply(str.upper())
sale_df['item_name'] = sale_df['item_name'].str.upper()
sale_df['item_name'].head()

sale_df['item_name'] = sale_df['item_name'].str.replace(' ', '')
sale_df['item_name'].head()

sale_df['item_name'].unique()
# len(sale_df['item_name'].unique())
sale_df['item_name'].nunique()

sale_df.sort_values(by= 'item_name', ascending= True)

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1748,2019-05-19 20:22:00,상품A,100.0,김시훈,201905
223,2019-06-25 08:13:00,상품A,100.0,김유진,201906
1742,2019-06-13 16:03:00,상품A,100.0,김건희,201906
1738,2019-02-10 00:28:00,상품A,100.0,김하랑,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:00,상품Y,NaN,김동욱,201904
2881,2019-04-30 14:21:00,상품Y,NaN,김하준,201904
1525,2019-01-24 10:27:00,상품Y,2500.0,김범준,201901
1361,2019-05-28 13:45:00,상품Y,2500.0,김수현,201905


In [460]:
for item_nm in sale_df['item_name'].sort_values().unique() :
  max_price = sale_df.loc[(sale_df['item_name'] == item_nm), 'item_price'].max()
  min_price = sale_df.loc[(sale_df['item_name'] == item_nm), 'item_price'].min(skipna = False)

  if max_price == min_price :
    print(item_nm, ':', max_price)
  
  else :
    print('상품 가격 오류 > ', item_nm, ':', max_price, '!=', min_price)

상품 가격 오류 >  상품A : 100.0 != nan
상품 가격 오류 >  상품B : 200.0 != nan
상품 가격 오류 >  상품C : 300.0 != nan
상품 가격 오류 >  상품D : 400.0 != nan
상품 가격 오류 >  상품E : 500.0 != nan
상품 가격 오류 >  상품F : 600.0 != nan
상품 가격 오류 >  상품G : 700.0 != nan
상품 가격 오류 >  상품H : 800.0 != nan
상품 가격 오류 >  상품I : 900.0 != nan
상품 가격 오류 >  상품J : 1000.0 != nan
상품 가격 오류 >  상품K : 1100.0 != nan
상품 가격 오류 >  상품L : 1200.0 != nan
상품 가격 오류 >  상품M : 1300.0 != nan
상품 가격 오류 >  상품N : 1400.0 != nan
상품 가격 오류 >  상품O : 1500.0 != nan
상품 가격 오류 >  상품P : 1600.0 != nan
상품 가격 오류 >  상품Q : 1700.0 != nan
상품 가격 오류 >  상품R : 1800.0 != nan
상품 가격 오류 >  상품S : 1900.0 != nan
상품 가격 오류 >  상품T : 2000.0 != nan
상품 가격 오류 >  상품U : 2100.0 != nan
상품 가격 오류 >  상품V : 2200.0 != nan
상품 가격 오류 >  상품W : 2300.0 != nan
상품 가격 오류 >  상품X : 2400.0 != nan
상품 가격 오류 >  상품Y : 2500.0 != nan
상품Z : 2600.0


>> 금액 결측치 수정

In [461]:
# 연속형

sale_df['item_price'].head()
sale_df['item_price'].describe()
sale_df.isnull().any(axis= 0)


# 이번 케이스의 경우 집계 기간에 상품 단가의 변동이 없다는 전제 조건이 있었기 때문에
# 결측치는 같은 상품의 단가를 이용하면 수정할 수 있습니다.


purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

In [462]:
import numpy as np

item_name_price_dict = dict()

item_name_price_dict['상품A']  = int(np.float64(10000))
item_name_price_dict

{'상품A': 10000}

In [463]:
# 실습)

item_name_price_dict = dict()

for idx, item_nm in enumerate(sale_df['item_name']) :
  # print(idx, ':', item_nm) 
  if pd.notna(sale_df.loc[idx, 'item_price']) :
    item_name_price_dict[item_nm] = int(sale_df.loc[idx, 'item_price'])
  
for idx, item_nm in enumerate(sale_df['item_name']) :
  if pd.isna(sale_df.loc[idx, 'item_price']) :
    sale_df.loc[idx, 'item_price'] = item_name_price_dict[item_nm]

sale_df.isnull().any(axis= 0)
sale_df['item_price'].head()

0     100.0
1    1900.0
2     100.0
3    2600.0
4     100.0
Name: item_price, dtype: float64

In [464]:
# 책)

is_null_df = sale_df['item_price'].isnull()
is_null_df.head()

sale_df.loc[is_null_df, 'item_name'].unique()
len(sale_df.loc[is_null_df, 'item_name'].unique()) 

# 따로 인덱스를 호출하지 않아도 ok (is_null_df.index)
# is_null_df.index를 따로 쓰지 않아도 되는 이유는,

# is_null_df 자체가 sale_df와 같은 인덱스를 가지고 있어서 "Boolean mask"로 작동하기 때문
# 만약, .index를 사용하고 싶다면
# sale_df.loc[is_null_df[is_null_df].index]

# loc[Boolean Series]가 훨씬 깔끔 & 직관적

for nan_item_nm in sale_df.loc[is_null_df, 'item_name'].unique() :
  price = sale_df.loc[(~is_null_df) & (sale_df['item_name'] == nan_item_nm), 'item_price'].max() # 특정한 값을 불러오기 위함
  sale_df.loc[(is_null_df) & (sale_df['item_name'] == nan_item_nm), 'item_price'] = price

sale_df.head()

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904


>> 금액 결측치 수정 점검

In [465]:
sale_df.isnull().any()
sale_df.isnull().sum()

for item_nm in sale_df['item_name'].sort_values().unique() :
  # print(item_nm)
  max_price = sale_df.loc[sale_df['item_name'] == item_nm, 'item_price'].max()
  min_price = sale_df.loc[sale_df['item_name'] == item_nm, 'item_price'].min(skipna = False)
  if max_price == min_price :
    print(item_nm, '가격', max_price)
  else :
    print(item_nm, '의 최고가 : ', max_price)
    print(item_nm, '의 최저가 : ', min_price)


상품A 가격 100.0
상품B 가격 200.0
상품C 가격 300.0
상품D 가격 400.0
상품E 가격 500.0
상품F 가격 600.0
상품G 가격 700.0
상품H 가격 800.0
상품I 가격 900.0
상품J 가격 1000.0
상품K 가격 1100.0
상품L 가격 1200.0
상품M 가격 1300.0
상품N 가격 1400.0
상품O 가격 1500.0
상품P 가격 1600.0
상품Q 가격 1700.0
상품R 가격 1800.0
상품S 가격 1900.0
상품T 가격 2000.0
상품U 가격 2100.0
상품V 가격 2200.0
상품W 가격 2300.0
상품X 가격 2400.0
상품Y 가격 2500.0
상품Z 가격 2600.0


>> 고객 이름 오류 수정

In [466]:
sale_df['customer_name'].head()

0    김가온
1    김우찬
2    김유찬
3    김재현
4    김강현
Name: customer_name, dtype: object

In [467]:
customer_df['고객이름'].head()

0    김 현성
1    김 도윤
2    김 지한
3    김 하윤
4    김 시온
Name: 고객이름, dtype: object

In [468]:
customer_df['고객이름'] = customer_df['고객이름'].str.replace(' ', '')
customer_df['고객이름'].head()

# 같은 이름의 데이터가 존재하는 경우 등록일, 생년월일 등 다른 정보를 이용해서 구분

0    김현성
1    김도윤
2    김지한
3    김하윤
4    김시온
Name: 고객이름, dtype: object

>> 날짜 오류 수정

In [469]:
customer_df['등록일'].head()
# xlsx 원본 파일 확인

0    2018-01-04 00:00:00
1                  42782
2    2018-01-07 00:00:00
3                  42872
4                  43127
Name: 등록일, dtype: object

In [470]:
# customer_df['등록일'] = pd.to_datetime(customer_df['등록일'])
# customer_df['등록일'].head()

digit_date = customer_df['등록일'].astype('str').str.isdigit()
digit_date.sum()

digit_to_datetime = pd.to_timedelta(customer_df.loc[digit_date, '등록일'].astype('float'), unit= 'D') + pd.to_datetime('1900-01-01')
str_to_datetime = pd.to_datetime(customer_df.loc[~digit_date, '등록일'])

print(digit_to_datetime)
print(str_to_datetime)

error_to_datetime = pd.concat([digit_to_datetime, str_to_datetime]) # index를 유지하고 있음
print(error_to_datetime)

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 등록일, dtype: datetime64[ns]
0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]
1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 200, dtype: datetime64[ns]


In [471]:
customer_df['등록일'] = error_to_datetime
customer_df['등록일']

0     2018-01-04
1     2017-02-18
2     2018-01-07
3     2017-05-19
4     2018-01-29
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 200, dtype: datetime64[ns]

In [472]:
# 등록일로부터 등록월을 추출해서 집계
customer_df['등록월'] = customer_df['등록일'].dt.strftime('%Y%m')
customer_df['등록월']

res = customer_df.groupby(by= '등록월')['고객이름'].count()
res
# len(customer_df)
# res.sum()

등록월
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64

In [473]:
# 등록일 처리 잘 되었는지 확인
is_digit_date = customer_df['등록일'].astype('str').str.isdigit()
is_digit_date.sum()

np.int64(0)

>> 고객 이름을 키로 두 개의 데이터를 결합(조인)

1장에서는 정합성이 있는 데이터였기 때문에 ID와 같은 공통키로 결합할 수 있었지만,           
여기서는 두 개의 데이터를 서로 다른 열을 지정해서 결합해야 합니다.

In [474]:
print(sale_df.head())
print(customer_df.head())

        purchase_date item_name  item_price customer_name purchase_month
0 2019-06-13 18:02:00       상품A       100.0           김가온         201906
1 2019-07-13 13:05:00       상품S      1900.0           김우찬         201907
2 2019-05-11 19:42:00       상품A       100.0           김유찬         201905
3 2019-02-12 23:40:00       상품Z      2600.0           김재현         201902
4 2019-04-22 03:09:00       상품A       100.0           김강현         201904
  고객이름  지역        등록일     등록월
0  김현성  H시 2018-01-04  201801
1  김도윤  E시 2017-02-18  201702
2  김지한  A시 2018-01-07  201801
3  김하윤  F시 2017-05-19  201705
4  김시온  E시 2018-01-29  201801


In [475]:
merge_df = sale_df.merge(right= customer_df, how= 'left', left_on= 'customer_name', right_on= '고객이름')
merge_df.head()

drop_merge_df = merge_df.drop(labels= 'customer_name', axis= 1)
drop_merge_df.head()


,purchase_date,item_name,item_price,purchase_month,고객이름,지역,등록일,등록월
0,2019-06-13 18:02:00,상품A,100.0,201906,김가온,C시,2017-01-26,201701
1,2019-07-13 13:05:00,상품S,1900.0,201907,김우찬,C시,2018-04-07,201804
2,2019-05-11 19:42:00,상품A,100.0,201905,김유찬,A시,2018-06-19,201806
3,2019-02-12 23:40:00,상품Z,2600.0,201902,김재현,D시,2018-07-22,201807
4,2019-04-22 03:09:00,상품A,100.0,201904,김강현,D시,2017-06-07,201706


>> 정제된 데이터의 덤프(출력)

깨끗해진 데이터를 파일로 출력(덤프)해두고 분석할 때 출력한 파일을 다시 읽어 들이면 데이터 정제를 다시 할 필요가 없습니다.

In [476]:
# 정제된 데이터 칼럼의 순서
# purchase_date > item_name > item_price > purchase_month > 고객이름 > 지역 > 등록일 > 등록월

# > purchase_date와 purchase_month는 가까이 있는 편이 다루기 용이

dump_data = merge_df[['purchase_date', 'purchase_month', 'item_name', 'item_price',
                      '고객이름', '지역', '등록일']]
dump_data.to_csv('./data/2장/dump_data.csv', index= False)


In [477]:
# 데이터의 변경이 있는 경우에는 다시 데이터 가공 처리

In [478]:
dump_data = pd.read_csv('./data/2장/dump_data.csv')
dump_data

,purchase_date,purchase_month,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07
...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,201902,상품Y,2500.0,김정민,B시,2017-07-01
2995,2019-06-22 04:03:00,201906,상품M,1300.0,김재원,E시,2018-03-31
2996,2019-03-29 11:14:00,201903,상품Q,1700.0,김지율,B시,2017-03-15
2997,2019-07-14 12:56:00,201907,상품H,800.0,김승주,E시,2018-07-15


In [479]:
# purchase_month를 세로축으로 상품별 집계

by_product = dump_data.pivot_table(index= 'purchase_month', columns= 'item_name',
                                       aggfunc= 'size', fill_value= 0)
by_product

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


In [480]:
# purchase_month를 세로축으로 매출 금액, 고객, 지역을 집계
# 1) 매출 금액
by_price = dump_data.pivot_table(values= 'item_price', index= 'purchase_month', columns= 'item_name',
                                 aggfunc= 'sum', fill_value= 0)
by_price

# 2) 고객
by_customer = dump_data.pivot_table(index= 'purchase_month', columns= '고객이름', aggfunc= 'size', fill_value= 0)
by_customer

# 3) 지역
by_region = dump_data.pivot_table(index= 'purchase_month', columns= '지역', aggfunc= 'size', fill_value= 0)
by_region

지역,A시,B시,C시,D시,E시,F시,G시,H시
purchase_month,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


In [488]:
# 집계 기간에 구매 이력이 없는 사용자 확인
# 기존 dump_data는 구매 이력이 있는 사용자만 확인

away_data = pd.merge(left= sale_df, right= customer_df, how= 'right', right_on= '고객이름', left_on= 'customer_name')
away_data

drop_away_data = away_data.drop(labels= 'customer_name', axis= 1)
drop_away_data

# 1) 실습
purchase_null_data = drop_away_data['purchase_month'].isnull()
purchase_null_customer = drop_away_data.loc[purchase_null_data, '고객이름']
purchase_null_customer

# 2) 책
purchase_null_customer = drop_away_data[drop_away_data['purchase_month'].isnull()][['고객이름', '등록일']]
purchase_null_customer

,고객이름,등록일
2999,김서우,2019-04-23


In [ ]:
# 고객 대장을 기준으로 매출 이력을 오른쪽 조인해서 결합
# 집계 기간 내 구매하지 않은 고객은 purchase 데이터가 NaN으로 채워짐
# > 집계 기간에 구매 이력이 없는 고객 조사

날짜 오류 수정에서 엑셀에서 읽어 들인 날짜 데이터를 수정하면 파이썬에서 '이틀' 어긋나게 변환

엑셀)
1. 엑셀 데이터의 출발 숫자는 0이 아닌 1부터 시작
2. 1900년은 평년이지만 엑셀에서 1900/02/29를 유효한 날짜로 계산

파이썬)
1. 파이썬 데이터의 출발 숫자는 0
2. 1900년은 평년이어서 1900년 2월 29일이라는 날짜는 없으며 파이썬에서도 카운트되지 않음

> 따라서 엑셀의 날짜 형식의 숫자를 단순히 파이썬으로 계산하면 이틀이 어긋남             
> float_date = pd.to_timedelta(customer_df.loc[float_date, '등록일'].astype(float) - 2, unit= 'D') + pd.to_datetime('1900/1/1')
